# Playground

A safe space for experimentation.

In [4]:
import requests
from etl_task import ETLTask
from utility import base64encode_obj

In [6]:
def load_data_bikes():
    """ Loads last 30 min snapshot of dublin bike stands. """
    import requests
    data = []
    try:
        res = requests.get(f"https://data.smartdublin.ie/dublinbikes-api/last_snapshot")  
        data = res.json()
    except Exception as e:
          data = []
          print(f'Failed to fetch dublin bikes data from source: {e}')
          raise Exception(f'Failed to fetch dublin bikes data from source: {e}')
    return data

def transform_data_bikes(data):
    """
    Transforms bikes data to be in a desireable format for saving.
    @param data: Data to be transformed.
    """
    import pandas as pd

    df = pd.DataFrame(data)
    df['usage_percent'] = df['available_bikes']/df['bike_stands']
    df['usage_percent'] = df['usage_percent'].round(2)
    df['status'] = df['status'].str.lower()
    df = df[[
        'station_id', 'bike_stands', 
        'available_bikes', 'usage_percent', 
        'last_update', 'status'
    ]]
    print("Bike data transformed. Usage % computed and last update made lowercase.")

    return df.to_dict(orient='records')

def save_data_bikes(data):
    """ 
    Saves given data to csv file. 
    @param data: Data to be saved.
    """
    import requests
    try:
        res = requests.post(
            url="http://localhost:8000/bikes/snapshot/", 
            json={'snapshot':data}
        )
        print(res.json()['message'])
    except Exception as e:
        print(f'Failed to save bikes data. {e}')
        raise Exception(f'Failed to save bikes data. {e}')
    
# data = load_data_bikes()
# data = transform_data_bikes(data)
# save_data_bikes(data)

In [7]:
# Toy data load and save functions.
def load_toy_data(): 
    """ A toy placeholder load data function. """
    print(f'Loaded data.')
    return 1

def transform_toy_data(data):
    """ A toy placeholder transform data function. """
    data_transformed = data * 10 # Deliberate error to showcase logging.
    print(f'Transformed data {data} into {data_transformed}.')
    return data_transformed

def save_toy_data(data): 
    """ A toy placeholder save data function. """
    import os
    import csv
    from datetime import datetime
    csv_file = './data.csv'
    file_exists = os.path.isfile(csv_file)
    with open(csv_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(['timestamp', 'description'])
        writer.writerows([[str(datetime.now()), "toy data"]])
    print(f'Saved data {data}.')

In [8]:
def make_post_request(url, data={}):
    """
    Makes a post request.
    @param url: URL to post to.
    @data: Request body data.
    @return: Response.
    """
    response = requests.post(url, params=data)
    return {'status': response.status_code, 'message': response.json()['message']}

def make_get_request(url, data={}):
    """
    Makes a post request.
    @param url: URL to post to.
    @data: Request body data.
    @return: Response.
    """
    response = requests.get(url, params=data)
    return {'status': response.status_code, 'message': response.text}

def make_delete_request(url, data={}):
    """
    Makes a delete request.
    @param url: URL to post to.
    @data: Request body data.
    @return: Response.
    """
    response = requests.delete(url, params=data)
    return {'status': response.status_code, 'message': response.json()['message']}

def make_put_request(url, task_name, new_values):
    """
    Makes a put request.
    @param url: URL to post to.
    @data: Request body data.
    @return: Response.
    """
    response = requests.put(
        url=url, 
        params={"task_name": task_name}, 
        json=new_values
    )
    return {'status': response.status_code, 'message': response.json()}

In [23]:
# Create a new task.
task = ETLTask(
    name="task1",
    fun_data_load=load_toy_data,
    fun_data_transform=transform_toy_data,
    fun_data_save=save_toy_data,
    repeat_time_unit='seconds',
    repeat_interval=5
)
task_str = base64encode_obj(task)
response = requests.post("http://127.0.0.1:8003/task", params={"task_str": task_str})

In [22]:
# # Shutdown the server.
# try:
#     requests.get("http://127.0.0.1:8003/end")
# except ConnectionResetError:
#     print("Server has been shut down.")

In [13]:
# Delete a task.
make_delete_request(url="http://127.0.0.1:8003/task", data={'task_name': 'task1'})

{'status': 200,
 'message': 'Success. If the task "task1" existed, it has been deleted.'}

In [10]:
# # Start scheduler.
# make_get_request(url="http://127.0.0.1:8003/start_scheduler")

In [11]:
# # Stop scheduler.
# make_get_request(url="http://127.0.0.1:8003/stop_scheduler")

In [12]:
# # Stop task
# make_put_request(
#     url="http://127.0.0.1:8003/task/stop", 
#     task_name="task1",
#     new_values={}
# )

In [13]:
# # Start task
# make_put_request(
#     url="http://127.0.0.1:8003/task/start", 
#     task_name="bikes",
#     new_values={}
# )